In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#impoer selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import time

In [9]:
base_url = 'https://www.booking.com'
attractions_url = base_url + "/attractions/index.en-gb.html"
attractions_url

'https://www.booking.com/attractions/index.en-gb.html'

In [10]:
# load url in selenium
driver = webdriver.Chrome()
driver.get(attractions_url)

In [11]:
wait = WebDriverWait(driver, 10)

In [12]:
see_more = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.css-1kke6ep .b4db3523c5.a861df9be1 button.a83ed08757.f3441ccb97.ec66406250[aria-controls="1"]')))
driver.execute_script("arguments[0].scrollIntoView();", see_more)
time.sleep(2)  
driver.execute_script("arguments[0].click();", see_more)
time.sleep(10)

In [13]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [14]:
destinations = soup.find_all('li', class_='css-11mgtht')
destinations[:2]

[<li class="css-11mgtht"><div class="css-v2kfba"><div class="intersection-visible-wrapper"><a class="css-1o507bj" href="/attractions/searchresults/us/new-york.en-gb.html?source=attractions_index_open_shop" title="New York"><div class="css-dz1uy1"><img alt="" class="css-3ilzvz" src="https://q-xx.bstatic.com/xdata/images/city/533x300/977435.jpg?k=ed701a5e2548cde3b7b819c951e57aa6529d9359bf1cd8a8cf1de57a78c2c122&amp;o="/></div><div class="c651ca389e fa2bb6146b"><div class="ba9417274c"><div class="a089bea0a0"><div class="bcdcb105b3">New York</div><div class="abf093bdfe">1344 things to do</div></div></div></div></a></div></div></li>,
 <li class="css-11mgtht"><div class="css-v2kfba"><div class="intersection-visible-wrapper"><a class="css-1o507bj" href="/attractions/searchresults/us/las-vegas.en-gb.html?source=attractions_index_open_shop" title="Las Vegas"><div class="css-dz1uy1"><img alt="" class="css-3ilzvz" src="https://q-xx.bstatic.com/xdata/images/city/533x300/977412.jpg?k=1b13db0f4340c5e

In [15]:
destinations[0].find('a')['href']

'/attractions/searchresults/us/new-york.en-gb.html?source=attractions_index_open_shop'

In [16]:
destinations_urls = []
for destination in destinations:
    destinations_urls.append({
        'name': destination.find('div', class_='bcdcb105b3').text,
        'link' : destination.find('a')['href']
    })

In [17]:
len(destinations_urls)

77

In [24]:
destinations_urls[:2]

[{'name': 'New York',
  'link': '/attractions/searchresults/us/new-york.en-gb.html?source=attractions_index_open_shop'},
 {'name': 'Las Vegas',
  'link': '/attractions/searchresults/us/las-vegas.en-gb.html?source=attractions_index_open_shop'}]

In [25]:
# convert destinations_urls to a dataframe
df_destinations = pd.DataFrame(destinations_urls)
df_destinations.head()

,name,link
0,New York,/attractions/searchresults/us/new-york.en-gb.h...
1,Las Vegas,/attractions/searchresults/us/las-vegas.en-gb....
2,Key West,/attractions/searchresults/us/key-west.en-gb.h...
3,San Diego,/attractions/searchresults/us/san-diego.en-gb....
4,Miami,/attractions/searchresults/us/miami.en-gb.html...


In [27]:
df_destinations.to_csv('booking_urls.csv', index=False)

# Scrape individual locations

In [34]:
df = pd.read_csv('booking_urls.csv')
df.head()

,name,link
0,New York,/attractions/searchresults/us/new-york.en-gb.h...
1,Las Vegas,/attractions/searchresults/us/las-vegas.en-gb....
2,Key West,/attractions/searchresults/us/key-west.en-gb.h...
3,San Diego,/attractions/searchresults/us/san-diego.en-gb....
4,Miami,/attractions/searchresults/us/miami.en-gb.html...


In [35]:
def scrape_single_attraction(soup):
    name = soup.find('h3', class_='css-33m8bc')
    name = name.text if name is not None else ""
    description = soup.find('div', class_='css-6k49yo css-6k49yo')
    description = description.text if description is not None else ""
    duration = soup.find('div', class_='a53cbfa6de css-j786b1')
    duration = duration.text if duration is not None else ""
    rating = soup.find('span', class_='a53cbfa6de css-35ezg3')
    rating = rating.text if rating is not None else ""
    reviews = soup.find('span', class_='a53cbfa6de')
    reviews = reviews.text if reviews is not None else ""
    price = soup.find('div', class_='e1eebb6a1e css-13pzcpe')
    price = price.text if price is not None else ""
    link = soup.find('a', class_='css-i6rjpg')
    link = link.href if link is not None else ""
    
    attraction = {
        'name': name,
        'description': description,
        'duration': duration,
        'rating': rating,
        'reviews': reviews,
        'price': price,
        'link': link,
    }
    print(attraction)
    return attraction

In [36]:
def scrape_attractions_city(url):
    driver.get(url)
    
    time.sleep(5)
    
    wait = WebDriverWait(driver, 20)

    # while True:
    #     try:
    #         see_more = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[class="a83ed08757 c21c56c305 bf0537ecb5 f671049264 deab83296e af7297d90d"]')))
    #         driver.execute_script("arguments[0].scrollIntoView();", see_more)
    #         time.sleep(2)  
    #         driver.execute_script("arguments[0].click();", see_more)
    #         time.sleep(10) 
    #         continue
    #     except TimeoutException:
    #         print('No more button to show')
    #         break
    #     except Exception as e:
    #         print(e)
    #         break
        
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    cards = soup.find_all('div', class_='b817090550 b736e9e3f4')
    city_attractions = []
    for card in cards:
        attraction = scrape_single_attraction(card)
        city_attractions.append(attraction)
    
    return city_attractions


In [37]:
attractions = []
for destination_url in df['link'][1]:
    city_url = base_url + destination_url
    attraction = scrape_attractions_city(city_url)
    attractions.append(attraction)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.87)
Stacktrace:
0   chromedriver                        0x00000001012e441c chromedriver + 4326428
1   chromedriver                        0x00000001012dc8e4 chromedriver + 4294884
2   chromedriver                        0x0000000100f08088 chromedriver + 278664
3   chromedriver                        0x0000000100ee3270 chromedriver + 127600
4   chromedriver                        0x0000000100f6f9ac chromedriver + 702892
5   chromedriver                        0x0000000100f82c0c chromedriver + 781324
6   chromedriver                        0x0000000100f3f4e4 chromedriver + 505060
7   chromedriver                        0x0000000100f3ff5c chromedriver + 507740
8   chromedriver                        0x00000001012a79b8 chromedriver + 4078008
9   chromedriver                        0x00000001012ac770 chromedriver + 4097904
10  chromedriver   

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
cards = soup.find_all('div', class_='b817090550 b736e9e3f4')
cards[0].find('h3', class_='css-33m8bc').text

'Historical Walking Tour, Discover and Explore San Miguel'

In [ ]:
len(attractions)

77

In [ ]:
df_list = pd.DataFrame(attractions)

In [ ]:
df_list.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,"{'name': 'SUMMIT One Vanderbilt Tickets', 'des...","{'name': '9/11 Memorial & Museum Admission', '...","{'name': 'New York CityPASS', 'description': '...","{'name': 'SUMMIT One Vanderbilt Ticket', 'desc...","{'name': 'One-hour Sightseeing Yacht Cruise', ...",{'name': 'New York City Statue of Liberty Supe...,"{'name': 'Empire State Building Observatory', ...",{'name': 'Admission to The Museum of Modern Ar...,"{'name': 'New York Explorer Pass', 'descriptio...","{'name': 'Broadway Tickets to The Lion King', ...",...,"{'name': 'Museum of Modern Art Ticket', 'descr...",{'name': 'Walking tour in Manhattan Brooklyn B...,"{'name': 'Aladdin Musical Admission', 'descrip...",{'name': 'Brooklyn Nets Basketball Game Ticket...,{'name': 'Circle Line Statue of Liberty Cruise...,{'name': 'Statue of Liberty and Ellis Island T...,{'name': 'NY 60 min Statue of Liberty Sightsee...,"{'name': '9/11 Tribute Museum and Memorial', '...","{'name': 'Bus Tour of Brooklyn, the Bronx and ...","{'name': 'Guided City Helicopter Tour', 'descr..."
1,{'name': 'Las Vegas Strip Helicopter Ride at N...,{'name': 'High Roller Observation Wheel Admiss...,{'name': 'Tickets to Mystère™ by Cirque du Sol...,{'name': 'Grand Canyon West Rim Tour & Hoover ...,{'name': 'Grand Canyon West Rim Bus Tour from ...,{'name': 'The STRAT Tower Observation Decks Ad...,{'name': '‘Rogue: The Sexiest Show in Vegas’ T...,{'name': 'Grand Canyon West Rim Tour with Brea...,"{'name': 'Three-hour Hoover Dam Guided Tour', ...",{'name': 'Paris Las Vegas Eiffel Tower Observa...,...,"{'name': 'iLuminate Show Ticket', 'description...","{'name': 'Madame Tussauds Las Vegas Ticket', '...",{'name': 'Morning Maverick Horseback Ride with...,{'name': 'America’s Got Talent presents Super ...,"{'name': 'The STRAT: SkyPod, SkyJump and Thril...",{'name': 'Attractions of the Red Rock Canyon T...,"{'name': 'Death Valley Tour from Las Vegas', '...",{'name': 'FlyOver Las Vegas Simulated Flight R...,{'name': 'Mad Apple by Cirque du Soleil Admiss...,{'name': 'Las Vegas Hip Hop Club Tour w/ Party...
2,{'name': 'Snorkelling Tour with Breakfast and ...,"{'name': 'Key West Sunset Cruise', 'descriptio...","{'name': 'Haunted Bar Crawl & Ghost Tour', 'de...",{'name': 'Key West Shark and Wildlife Viewing ...,{'name': 'Dolphin-watching Cruise with Snorkel...,{'name': 'Key West Dolphin Watching & Snorkell...,{'name': 'Mangrove Kayak Eco Tour in Key West'...,{'name': 'Key West Sunset Sailing Tour with Ba...,{'name': 'Glass-bottom Boat Tour of the Florid...,{'name': 'Sunset Sailing Tour with Drinks and ...,...,"{'name': 'Parasailing Adventure in Key West', ...","{'name': 'Key West Eco Tour with Snorkelling',...","{'name': 'Key West Food Tour', 'description': ...",{'name': 'The Original Key West Island Jet Ski...,{'name': 'Dolphin Watching and Snorkeling Adve...,"{'name': 'Key West Half-day Kayak Rental', 'de...",{'name': 'Self-guided Tour of Key West’s Old T...,{'name': 'Private Sunset Cruise around Key Wes...,"{'name': 'Two-stop Coral Reef Snorkel Trip', '...","{'name': 'Drop Off Sportfishing - Key West', '..."
3,"{'name': 'San Diego Zoo Day Pass', 'descriptio...",{'name': 'USS Midway Museum Admission with Aud...,"{'name': 'San Diego Harbour Cruise', 'descript...","{'name': 'San Diego Harbour Cruise', 'descript...","{'name': 'San Diego Zoo', 'description': '', '...","{'name': 'Whale-watching Cruise', 'description...","{'name': 'USS Midway Museum Ticket', 'descript...","{'name': 'San Diego Whale Watching Cruise', 'd...",{'name': 'One-day Pass to San Diego Zoo Safari...,"{'name': 'Dolphin and Whale Sunset Cruise', 'd...",...,{'name': 'San Diego Father's Day Buffet Brunch...,{'name': 'Gondola Cruise through the Coronado ...,{'name': 'Admission to San Diego Zoo Safari Pa...,"{'name': 'Small-group Sunset Cruise', 'descrip...","{'name': 'Private Sunset Sail for 3-6 People',...",{'name': '4 Hour Private Sail for up to 6 peop...,"{'name': 'Walking Ghost Tour in San Diego', 'd..